# Auction Mechanism for conflict resolution
Author Ilias Parmaksizoglou & Alessandro Bombelli

This notebook contains a simple implementation of a VCG Auction for the case where Tool 3.1 assignment, exceeds demand. All stated input are arbitrary

In [31]:
import gurobipy as gp
from gurobipy import GRB

# Load Data

A simple case where three companies compete for 4 spots in total in a terminal with two destionations in total. C1 is interesetd in two spots, C2 in all 4 and C3 in only 1. All bid valuations and target destinations are generated arbitrarily

In [32]:
B, B_val = gp.multidict({
    ('C1',1): [2],
    ('C1',2): [3],
    ('C2',1): [2],
    ('C2',2): [4],
    ('C2',3): [5],
    ('C2',4): [6],
    ('C3',1): [1]
 })

C, y1_spots,y2_spots = gp.multidict({
    'C1': [1,1],
    'C2': [3,1],
    'C3': [0,1]
 })

# User Stated parameters
cap_s = 2 # Capacity of a single yard
cap_w = 4 # Capacity of terminal in total
yards = range(1,3) # 
spots = (range(1,cap_w+1)) # Maximum Available spots in total
companies = ["C1","C2","C3"]


In [33]:
# Create optimization model
model = gp.Model('vcg')

# Derived Variables
x = model.addVars(B,vtype=GRB.BINARY, name="x")
a = model.addVars(companies,yards,vtype=GRB.INTEGER, name="a")
d = model.addVars(yards,vtype=GRB.INTEGER, name="d")

Constraint 1 determines an upper bound for spots acquired by a company at a specific destination based on their demand and the number of spots relating to a specific bid.

In [34]:
# Constraint 1

for y in yards:
    for c in companies:
        if y == 1:
            model.addConstr(y1_spots[c]*sum(x[b] for b in B if b[0] == c)>= a[c,y])
        if y == 2:
            model.addConstr(y2_spots[c]*sum(x[b] for b in B if b[0] == c)>= a[c,y])

Constraints 2 defines a lower bound for the number of spots that a company can acquire, by forcing that if a bid has been accepted, then the number of spots acquired should be greater or equal to the number of spots that bid referred to.

In [35]:
for b in B:
    model.addConstr(b[1] - sum(a[b[0],y] for y in yards) <= cap_w*(1-x[b]))

Constraint set 3 imposes that each company can be rewarded, at most, one winning bid. Constraints 4 - 5 define the number of spots allocated per destination during the time-window and the number of spots for the port in total.

In [36]:
# Constraint 3
for c in companies:
    model.addConstr(sum(x[b] for b in B if b[0] == c)<=1)

# Constraint 4
for y in yards:
    model.addConstr(sum(a[c,y] for c in companies) <=cap_s)

# Constraint 5
model.addConstr(sum(a[c,y] for c in companies for y in yards) <=cap_w)

<gurobi.Constr *Awaiting Model Update*>

The Objective function maximizes the social welfare of the system based on the provided bids by agents.

In [ ]:
# Solve The Model
#model.setParam("LogToConsole",0)
model.setObjective(sum(x[b]*B_val[b] for b in B), GRB.MAXIMIZE)
model.optimize()

a = []
for b in B:
    if x[b].x >=0.99:
        print(f"Company {b[0]} won {b[1]} spots")
   
